# 数据清洗

In [14]:
import numpy as np
import pandas as pd

### 税收不遵从

In [15]:
deregulation = pd.concat([
    pd.read_excel('RESSET_DEREGULATION_EXT_1.xls', engine='xlrd'),
    pd.read_excel('RESSET_DEREGULATION_EXT_2.xls', engine='xlrd')
], ignore_index=True)
deregulation['year'] = deregulation['信息发布日期_InfoPubDt'].dt.year - 1


In [16]:
# 检查处理原因和处分措施中是否包含关键词
pattern = '|'.join(['税'])
tax_noncomp = deregulation[
    deregulation['涉及法规_InvolStatute'].astype(str).str.contains(pattern, regex=True) # |
    # deregulation['处分措施_Pena'].astype(str).str.contains(pattern, regex=True)
]
tax_noncomp = tax_noncomp.drop_duplicates(subset=['year', '上市公司代码_ComCd'])[['上市公司代码_ComCd','最新公司全称_LComNm', 'year']]
tax_noncomp['noncompliance'] = 1

### 财报指标

In [17]:
finratio = pd.concat([
    pd.read_excel('RESSET_FINRATIO_1.xls', engine='xlrd'),
    pd.read_excel('RESSET_FINRATIO_2.xls', engine='xlrd'),
    pd.read_excel('RESSET_FINRATIO_3.xls', engine='xlrd')
], ignore_index=True)
finratio = finratio.rename(columns={'上市公司代码_Comcd': '上市公司代码_ComCd', 
                                    '最新公司全称_Lcomnm': '最新公司全称_LComNm'})
finratio['year'] = finratio['截止日期_Enddt'].dt.year


### 股权指标

In [18]:
share_type = pd.read_excel('RESSET_MSHLSTANDOS_1.xls', engine='xlrd')
share_type['year'] = share_type['截止日期_EndDt'].dt.year
share_type['股东类别_SHType'] = (share_type['股东类别编码_SHTypeCd'] == 10).astype(int)

share_num = pd.read_excel('RESSET_SHNUMBER_1.xls', engine='xlrd')
share_num['year'] = share_num['截止日期_EndDt'].dt.year

### 合并

In [ ]:
data = share_type[['上市公司代码_ComCd', '最新公司全称_LComNm', 'year']]
data = pd.merge(data,
                tax_noncomp[['上市公司代码_ComCd', 'year', 'noncompliance']],
                on=['year', '上市公司代码_ComCd'],
                how='left')
data['noncompliance'] = data['noncompliance'].fillna(0)
data = pd.merge(data, 
                share_type[['year', '上市公司代码_ComCd', '股东类别_SHType']],
                on=['year', '上市公司代码_ComCd'],
                how='left')
data = pd.merge(data, 
                share_num[['year', '上市公司代码_ComCd', '股东总户数(户)_SHNum', '户均持股数(股/户)_AvgHS']],
                on=['year', '上市公司代码_ComCd'],
                how='left')
data = pd.merge(data,
                finratio[[col for col in finratio.columns if col not in ['更新日期_Updt', '观测序号_nobs', '公司代码_CompanyCode', '截止日期_Enddt', '最新公司全称_LComNm']]],
                on=['year', '上市公司代码_ComCd'],
                how='left')

### 合规企业欠采样